# 模块 3：混合 RAG

我们已经在模块 1 中介绍了基础版的 RAG。在这里，我们将介绍一种新的 RAG 类型——混合 RAG。在某些场景下，我们希望获得更高的召回率，但仅靠嵌入向量检索无法获得理想的效果。

在这种情况下，通常会采用多通道检索来提升效果。除了嵌入向量检索外，另一个常用的通道是 BM25。

本模块将使用 Amazon Bedrock 嵌入模型和 TiDB Cloud Starter 向量检索。

> **注意：**
>
> - 我们已经在环境参数中设置了 `SERVERLESS_CLUSTER_HOST`、`SERVERLESS_CLUSTER_PORT`、`SERVERLESS_CLUSTER_USERNAME`、`SERVERLESS_CLUSTER_PASSWORD` 和 `SERVERLESS_CLUSTER_DATABASE_NAME`。
> - 我们还为本实验授予了 Amazon Bedrock 的使用权限。如果你想在 TiDB Labs 平台之外使用此代码片段，请提前进行相关设置。

## 安装依赖

In [ ]:
%pip install -q \
    pytidb==0.0.10.dev1 \
    boto3==1.38.23 \
    litellm \
    pandas

## 初始化数据库客户端

In [ ]:
import os

from litellm import completion
from typing import Optional, Any
from pytidb import TiDBClient
from pytidb.schema import TableModel, Field
from pytidb.embeddings import EmbeddingFunction

db = TiDBClient.connect(
    host=os.getenv("SERVERLESS_CLUSTER_HOST"),
    port=int(os.getenv("SERVERLESS_CLUSTER_PORT")),
    username=os.getenv("SERVERLESS_CLUSTER_USERNAME"),
    password=os.getenv("SERVERLESS_CLUSTER_PASSWORD"),
    database=os.getenv("SERVERLESS_CLUSTER_DATABASE_NAME"),
    enable_ssl=True,
)

embedding_model = "bedrock/amazon.titan-embed-text-v2:0"

text_embedding_function = EmbeddingFunction(
    embedding_model,
    timeout=60
)

## 准备上下文

和模块 1 类似，我们创建了 `documents` 表，但这次我们通过如下代码新增了一个索引：

```python
table.create_fts_index("text")
```

它会基于 `text` 列创建一个全文检索索引。

In [ ]:
table_name = "documents"
class Document(TableModel, table=True):
    __tablename__ = table_name
    __table_args__ = {"extend_existing": True}
    id: int | None = Field(default=None, primary_key=True)
    text: str = Field(max_length=1024)
    embedding: Optional[Any] = text_embedding_function.VectorField(
        source_field="text",
    )

documents = [
    Document(id=1, text="TiDB is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads."),
    Document(id=2, text="TiFlash is the key component that makes TiDB essentially an Hybrid Transactional/Analytical Processing (HTAP) database. As a columnar storage extension of TiKV, TiFlash provides both good isolation level and strong consistency guarantee."),
    Document(id=3, text="TiKV is a distributed and transactional key-value database, which provides transactional APIs with ACID compliance. With the implementation of the Raft consensus algorithm and consensus state stored in RocksDB, TiKV guarantees data consistency between multiple replicas and high availability. "),
]

table = db.create_table(schema=Document, if_exists="overwrite")

table.create_fts_index("text")

table.bulk_insert(documents)
table.query("ALTER TABLE documents COMPACT")

## 混合检索获取内容

这一次，我们不仅通过向量的余弦距离获取相关文档，还结合了 BM25 分数。然后我们使用融合方法合并结果集。

不过你无需手动实现，`pytidb` 已经帮你做好了。你只需要通过 `search_type="hybrid"` 并指定 `text_column` 即可检索，例如：



In [ ]:
question = "What is TiKV?"

results = table.search(question, search_type="hybrid").text_column("text").limit(1)
results.to_pandas()

## 生成答案

In [ ]:
from litellm import completion

llm_model = "bedrock/us.amazon.nova-lite-v1:0"

messages = [
    {"role": "system", "content": f"Please carefully answer the question by {str(results)}"},
    {"role": "user", "content": question}
]

llm_response = completion(
    model=llm_model,
    messages=messages,
)

print(llm_response.choices[0].message.content)